In [1]:
import boto3
import os
import logging
import pprint
import json
import pandas as pd

In [11]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='eu-central-1')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
dynamodb = boto3.resource('dynamodb', region_name='eu-central-1')

def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [12]:
%%time
import uuid

agent_id = 'XVOW2OYC6J'
alias_id = 'L12GCIKFOD'
session_id:str = str(uuid.uuid1())
query = "What is yolo?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

YOLO (You Only Look Once) is a popular object detection algorithm that frames object detection as a single regression problem. It uses a single neural network to predict bounding boxes and associated class probabilities for objects in an image in a single pass. The key idea behind YOLO is to divide the input image into an S x S grid, where each grid cell predicts B bounding boxes, confidence scores, and C class probabilities. The latest version, YOLOv8, introduces improvements such as a new neural network architecture that utilizes both Feature Pyramid Network (FPN) and Path Aggregation Network (PAN), as well as a new labeling tool to simplify the annotation process for training data. It also incorporates advanced post-processing techniques like Soft-NMS to refine detection results and improve accuracy.
CPU times: user 53.1 ms, sys: 3.23 ms, total: 56.3 ms
Wall time: 7.36 s
